In [4]:
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup
# import the library we use to open URLs
import urllib.request
# import pandas libbrary to load and manage data
import pandas as pd

In [5]:
# url of wikipeadia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

# find table
table = soup.find('table', class_='wikitable sortable')

# lists
A=[]
B=[]
C=[]

# fill lists
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

# assign list to dataframe
df_postcode = pd.DataFrame(A,columns=['Postcode'])
df_postcode['Borough']=B
df_postcode['Neighborhood']=C

df_postcode.head(20)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [19]:
# read csv file (geocoder not reliable)
df_latlon = pd.read_csv('http://cocl.us/Geospatial_data')

# join list of neighborhoods/postcodes with geospatial data
df_complete = pd.merge(df_postcode, df_latlon, left_on = 'Postcode', right_on = 'Postal Code')
# remove duplicate postcode column
df_complete.drop(columns=['Postal Code'], inplace=True)
df_complete

M3A
M4A
M5A
M5A
M6A
M6A
M7A
M9A
M1B
M1B
M3B
M4B
M4B
M5B
M5B
M6B
M9B
M9B
M9B
M9B
M9B
M1C
M1C
M1C
M3C
M3C
M4C
M5C
M6C
M9C
M9C
M9C
M9C
M1E
M1E
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M3H
M3H
M4H
M5H
M5H
M5H
M6H
M6H
M1J
M2J
M2J
M2J
M3J
M3J
M4J
M5J
M5J
M5J
M6J
M6J
M1K
M1K
M1K
M2K
M3K
M3K
M4K
M4K
M5K
M5K
M6K
M6K
M6K
M1L
M1L
M1L
M2L
M2L
M3L
M4L
M4L
M5L
M5L
M6L
M6L
M6L
M9L
M1M
M1M
M1M
M2M
M2M
M3M
M4M
M5M
M5M
M6M
M6M
M6M
M6M
M9M
M9M
M1N
M1N
M2N
M3N
M4N
M5N
M6N
M6N
M9N
M1P
M1P
M1P
M2P
M4P
M5P
M5P
M6P
M6P
M9P
M1R
M1R
M2R
M4R
M5R
M5R
M5R
M6R
M6R
M7R
M9R
M9R
M9R
M9R
M1S
M4S
M5S
M5S
M6S
M6S
M1T
M1T
M1T
M4T
M4T
M5T
M5T
M5T
M1V
M1V
M1V
M1V
M4V
M4V
M4V
M4V
M4V
M5V
M5V
M5V
M5V
M5V
M5V
M5V
M8V
M8V
M8V
M9V
M9V
M9V
M9V
M9V
M9V
M9V
M9V
M1W
M4W
M5W
M8W
M8W
M9W
M1X
M4X
M4X
M5X
M5X
M8X
M8X
M8X
M4Y
M7Y
M8Y
M8Y
M8Y
M8Y
M8Y
M8Y
M8Y
M8Y
M8Z
M8Z
M8Z
M8Z
M8Z


In [10]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [21]:
# map rendering library
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for index, row in df_complete.iterrows():
    label = '{}, {}'.format(row['Borough'], row['Neighborhood'])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row['Latitude'], row['Longitude']],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

# show map
map_toronto

In [25]:
toronto_data = df_complete[df_complete['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [33]:
toronto_data.head(20)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
5,M4E,East Toronto,The Beaches,43.676357,-79.293031
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M6G,Downtown Toronto,Christie,43.669542,-79.422564
9,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568


In [30]:
address = 'Downtown Toronto, Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


In [32]:
# map rendering library
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

# create map of Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for index, row in toronto_data.iterrows():
    label = '{}, {}'.format(row['Borough'], row['Neighborhood'])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row['Latitude'], row['Longitude']],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  

# show map
map_downtown_toronto